In [355]:
import iris

import numpy as np
import scipy.stats

In [356]:
%load_ext autoreload
%autoreload 2
import math_helpers

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [357]:
era5 = iris.load("testing_data/ERA5.nc")
obs = era5[1]

cm_hist = iris.load_cube(
    "testing_data/CMIP6/medium_area/historical_monthly_surface_temp_single_levels_IPSL-CM5A2-INCA/tas_Amon_IPSL-CM5A2-INCA_historical_r1i1p1f1_gr_19790116-20141216_v20200729.nc"
)
cm_future = iris.load_cube(
    "testing_data/CMIP6/medium_area/SSP3-7_monthly_surface_temp_single_levels_IPSL-CM5A2-INCA/tas_Amon_IPSL-CM5A2-INCA_ssp370_r1i1p1f1_gr_20150116-20501216_v20201218.nc"
)

/home/jakobwes/anaconda3/lib/python3.8/site-packages/iris/fileformats/cf.py:859: UserWarning: Missing CF-netCDF measure variable 'areacella', referenced by netCDF variable 'tas'
  warnings.warn(
/home/jakobwes/anaconda3/lib/python3.8/site-packages/iris/fileformats/cf.py:859: UserWarning: Missing CF-netCDF measure variable 'areacella', referenced by netCDF variable 'tas'
  warnings.warn(


In [358]:
# Regrid
cm_hist = cm_hist.regrid(obs, iris.analysis.Linear())
cm_future = cm_future.regrid(obs, iris.analysis.Linear())

In [359]:
# Get data
obs = np.array(obs.data[:, 0, :, :])
cm_hist = cm_hist.data
cm_future = cm_future.data

Test ISIMIP:

In [360]:
obs = obs[:, 1, 1]
cm_hist = cm_hist[:, 1, 1]
cm_future = cm_future[:, 1, 1]

In [361]:
from isimip import ISIMIP

temp_debiaser = ISIMIP(variable="tas")
temp_debiaser.variable = "prsnratio"
"""temp_debiaser.trend_preservation = "mixed"
temp_debiaser.lower_bound = 0
temp_debiaser.upper_bound = 300
temp_debiaser.lower_threshold = 50
temp_debiaser.upper_treshold = 290
temp_debiaser.powerlaw_exponent_step4 = 2"""

'temp_debiaser.trend_preservation = "mixed"\ntemp_debiaser.lower_bound = 0\ntemp_debiaser.upper_bound = 300\ntemp_debiaser.lower_threshold = 50\ntemp_debiaser.upper_treshold = 290\ntemp_debiaser.powerlaw_exponent_step4 = 2'

In [364]:
obs = obs[obs > 0]  # Two obs-outliers. Need to check for that.

In [365]:
obs, cm_hist, cm_future = temp_debiaser.step2(obs, cm_hist, cm_future)
obs, cm_hist, cm_future = temp_debiaser.step4(obs, cm_hist, cm_future)
obs_future = temp_debiaser.step5(obs, cm_hist, cm_future)
test = temp_debiaser.step6(obs, obs_future, cm_hist, cm_future)

In [366]:
x = temp_debiaser.apply_location(obs, cm_hist, cm_future)

[0, 400]


NameError: name 'variable' is not defined